# Задание

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.
Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

# Процессинг данных

Загрузим данные.     
Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [1]:
import csv

In [2]:
%%time
with open('checkins.dat') as input_file:        
    newLines = []
    for line in input_file:
        newLine = [x.strip() for x in line.split('|')]
        if len(newLine) == 6 and newLine[3] and newLine[4]:
            newLines.append(newLine)

Wall time: 3.5 s


In [4]:
%%time
with open('checkins.csv', 'w') as output_file:
    file_writer = csv.writer(output_file)
    file_writer.writerows(newLines)

Wall time: 779 ms


С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [5]:
import numpy as np
import pandas as pd

In [6]:
%%time
data = pd.read_csv('checkins.csv', header=0)

Wall time: 653 ms


In [7]:
len(data)

396634

In [8]:
data.head(10)

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18
5,984483,1030290,955969,32.221743,-110.926479,2012-04-21 17:58:54
6,984685,304253,23558,40.650000,-73.950000,2012-04-21 18:19:34
7,984470,720850,749715,33.448377,-112.074037,2012-04-21 17:02:47
8,984610,1639666,442605,33.414768,-111.909309,2012-04-21 18:04:58
9,984653,1647192,23558,42.358431,-71.059773,2012-04-21 18:23:22


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров. 
Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате: 38.8951118,-77.0363658

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса офисов: 33.751277, -118.188740 (Los Angeles) и т.д.

In [9]:
import sklearn.cluster as cluster

In [20]:
X = data.values[0:100000, 3:5]

In [23]:
cluster1 = cluster.MeanShift(bandwidth=0.1)

In [24]:
cluster1.fit(X)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [26]:
labels = cluster1.labels_
cluster_centers = cluster1.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)
n_clusters_

3230

In [30]:
cluster_centers[1]

array([  33.44943805, -112.00213969])

In [31]:
d = {}
for label in labels:
    if label not in d.keys():
        d[label] = 1
    else:
        d[label] += 1

In [33]:
count = 0
for key in d.keys():
    if d[key] > 15:
        count += 1
print(count)

593


In [34]:
clusters_select = np.ndarray(shape=(count,2))

In [35]:
clusters_select

array([[  3.93203654e-315,   3.05592566e-316],
       [  1.00131040e-307,   4.45061456e-308],
       [  1.33510679e-306,   6.23035620e-307],
       ..., 
       [  8.90077926e-308,   4.45061456e-308],
       [  7.78796646e-308,   1.22381779e-307],
       [  6.23035619e-307,   1.39071700e-307]])

In [36]:
i = 0
j = 0
while i < len(cluster_centers):
    if d[i] > 15:
        clusters_select[j] = cluster_centers[i]
        j += 1
    i += 1

In [37]:
len(clusters_select)

593

In [38]:
clusters_select[0]

array([ 40.7177164 , -73.99183542])

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу "Carnival Cruise Line" адреса офисов:   
33.751277, -118.188740 (Los Angeles)   
25.867736, -80.324116 (Miami)   
51.503016, -0.075479 (London)   
52.378894, 4.885084 (Amsterdam)   
39.366487, 117.036146 (Beijing)   
-33.868457, 151.205134 (Sydney)   
Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.   
Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [39]:
offices = np.ndarray(shape=(6,2))
offices[0] = np.array([33.751277, -118.188740])
offices[1] = np.array([25.867736, -80.324116])
offices[2] = np.array([51.503016, -0.075479])
offices[3] = np.array([52.378894, 4.885084])
offices[4] = np.array([39.366487, 117.036146])
offices[5] = np.array([-33.868457, 151.205134])

In [40]:
def distance(point1, point2):
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)**0.5

In [41]:
distance(offices[0], clusters_select[123])

20.892897123756782

In [42]:
answer_index = 0
min_dist = 0
i = 0
while i < len(clusters_select):
    distances = [distance(xx, clusters_select[i]) for xx in offices]
    if min_dist == 0:
        min_dist = min(distances)
        answer_index = i
    else:
        if min_dist > min(distances):
            min_dist = min(distances)
            answer_index = i
    i += 1

In [43]:
answer_index

410

In [44]:
min_dist

0.007834758163107856

In [45]:
clusters_select[answer_index]

array([ -33.86063043,  151.20477593])

In [46]:
def write_answer(center):
    with open("answer.txt", "w") as f:
        f.write(str(center[0]) + ' ' + str(center[1]))

In [47]:
write_answer(clusters_select[answer_index])